In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import gc
import xgboost as xgb
import math
from joblib import Parallel, delayed

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df: 
    @param threshold: 
    @return: 
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [3]:
# train_df = pd.read_hdf('../input/train.h5')
test_df = pd.read_hdf('../input/test.h5')
# sub = pd.DataFrame(test_df['time'])

In [4]:
# train_df = train_df[train_df['temperature'].notnull()]
# train_df = train_df.fillna(method='bfill')
test_df = test_df.fillna(method='bfill')
# gc.collect()

In [5]:
# train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
#                     'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [6]:
# print('train_df.shape: ', train_df.shape)
# train_df = train_df.loc[(train_df['outdoorTemp'] >= test_df['outdoorTemp'].min()) & (train_df['outdoorTemp'] <= test_df['outdoorTemp'].max())]
# print('处理后 train_df.shape: ', train_df.shape)

In [7]:
# y_train = train_df['temperature'] - train_df['outdoorTemp']
# y_train.to_hdf('../input/y_train.h5', 'df')

In [8]:
# del train_df, test_df
# gc.collect()

In [9]:
x_train = pd.read_hdf('../input/train_features.h5')
x_test = pd.read_hdf('../input/test_features.h5')
y_train = pd.read_hdf('../input/y_train.h5')

In [10]:
col_corr = correlation(x_train, 0.98)
# print(col_corr)

In [11]:
x_train.drop(list(col_corr), axis=1, inplace=True)
x_test.drop(list(col_corr), axis=1, inplace=True)

In [12]:
nums = int(x_train.shape[0] * 0.8)

trn_x, trn_y, val_x, val_y = x_train[:nums], y_train[:nums], x_train[nums:], y_train[nums:]

train_matrix = xgb.DMatrix(trn_x, label=trn_y, missing=np.nan)
valid_matrix = xgb.DMatrix(val_x, label=val_y, missing=np.nan)
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

params = {
    'booster': 'gbtree',
    'eval_metric': 'mae',
    'min_child_weight': 5,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.01,
    'seed': 2020,
    'nthread': 36,
    'silent': 1
}

watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

model_eval = xgb.train(params,
                       train_matrix,
                       num_boost_round=50000,
                       evals=watchlist,
                       verbose_eval=500,
                       early_stopping_rounds=1000)
val_pred = model_eval.predict(valid_matrix, ntree_limit=model_eval.best_ntree_limit).reshape(-1, 1)

[0]	train-mae:0.72620	eval-mae:0.31621
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 1000 rounds.
[500]	train-mae:0.04913	eval-mae:0.15253
[1000]	train-mae:0.03814	eval-mae:0.15209
[1500]	train-mae:0.03132	eval-mae:0.15193
[2000]	train-mae:0.02635	eval-mae:0.15211
Stopping. Best iteration:
[1319]	train-mae:0.03360	eval-mae:0.15176



In [13]:
mse = mean_squared_error(val_y, val_pred)
print("mse_score:", mse)
print("mse_score:", str(mse)[2:7])

mse_score: 0.037816330069916776
mse_score: 03781


In [14]:
feat_imp_dict = model_eval.get_score(importance_type='gain')

In [15]:
feat_imp = pd.Series(feat_imp_dict).sort_values(ascending=False).reset_index().rename(columns={'index': 'feature',
                                                                                               0: 'importance'})
feat_imp['normalized_importance'] = feat_imp['importance'] / feat_imp['importance'].sum()
feat_imp['cumulative_importance'] = np.cumsum(feat_imp['normalized_importance'])

feat_imp.to_csv('../input/feat_imp.csv', index=False)

In [16]:
record_low_importance = feat_imp[feat_imp['cumulative_importance'] > 0.92]
to_drop = list(record_low_importance['feature'])
print(to_drop)

['MD_indoorHum_diff_max_outdoorHum_multyply', 'MD_outdoorHum_medi_MDH_outdoorAtmo_min_ratio', 'outdoorTemp_diff_MDH_indoorHum_diff_max_multyply', 'MDH_indoorHum_diff_max_MDH_indoorAtmo_diff_sum_ratio', 'MD_outdoorHum_mean_MDH_outdoorAtmo_medi_subtract', 'MDH_outdoorAtmo_diff_sum_MD_outdoorHum_diff_sum_multyply', 'MDH_outdoorHum_diff_sum_MD_outdoorAtmo_diff_sum_ratio', 'MD_outdoorHum_min_MD_indoorHum_diff_max_ratio', 'MDH_indoorAtmo_diff_sum_MDH_indoorHum_diff_max_multyply', 'MD_indoorHum_diff_min_MDH_outdoorAtmo_diff_sum_multyply', 'MDH_outdoorTemp_diff_mean_MDH_indoorHum_diff_mean_multyply', 'MDH_indoorAtmo_diff_mean_MDH_outdoorAtmo_diff_mean_multyply', 'MD_outdoorAtmo_mean_MDH_indoorAtmo_medi_multyply', 'MDH_outdoorHum_diff_mean_MDH_outdoorAtmo_diff_mean_multyply', 'MDH_indoorAtmo_diff_sum_MDH_indoorHum_diff_sum_multyply', 'MDH_outdoorAtmo_diff_sum_MDH_indoorAtmo_min_multyply', 'MDH_outdoorAtmo_diff_sum_MD_outdoorAtmo_diff_sum_multyply', 'MDH_indoorHum_diff_max_MD_outdoorTemp_diff_me

In [17]:
x_train.drop(to_drop, axis=1, inplace=True)
x_test.drop(to_drop, axis=1, inplace=True)

In [18]:
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

model = xgb.train(params,
                  train_all_matrix,
                  num_boost_round=model_eval.best_ntree_limit + 20)

test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)

In [20]:
sub = pd.DataFrame(test_df['time'])
sub['temperature'] = test_pred[:, 0] + test_df['outdoorTemp'].values
sub.to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), str(mse)[2:7]), index=False)